# Quadrotor Ball Sim
by Sebastian Lopez-Cot

This python notebook can be used to run the quad-ball system. The quadrotor will try to send a ball to a desired location given initial conditions and will attempt to return to its original starting point.

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import scipy.interpolate
import matplotlib.pyplot as plt

from sim import ProjectileMath, TimeInterpolator, ContactingSwitch, Quadrotor, Ball, Animator
from quad_direct_transcription import QuadDirectTranscription

def plan_trajectory(init_quad_state, init_ball_state, ball_goal):
    # Given the ball's state, calculate it's arch info
    quad_target = np.zeros(6)
    arch_width, apex, travel_time, vland, theta = ProjectileMath.calc_arch_info(init_ball_state)
    print "arch_width: ", arch_width
    print "apex: ", apex
    print "travel_time: ", travel_time
    print "vland: ", vland
    print "theta: ", theta
    quad_target[0] = init_ball_state[0] + arch_width * np.sign(init_ball_state[2])
    quad_target[1] = init_ball_state[1]
    quad_target[2] = np.sign(theta) * (np.pi/2.0 - abs(theta))
    
    push_vel = 9.0
    norm = np.array([-np.sin(quad_target[2]), np.cos(quad_target[2])])
    quad_target[3:5] = push_vel * norm
    
    fall_time = travel_time
    return qdt.solve(init_quad_state, quad_target, fall_time)

quad = Quadrotor()
ball = Ball()
sw = ContactingSwitch()
qdt = QuadDirectTranscription(quad.m, quad.g)

dt = 0.0001
steps = 60000
downsample = 600

time = dt * np.arange(steps)

quad_state = np.zeros((steps, 6))
quad_state[0,0] = np.random.choice(np.linspace(-5, 5, 20))
quad_state[0,1] = np.random.choice(np.linspace(0, 5, 20))
quad_state[0,2] = np.random.choice(np.linspace(-np.pi/3.0, np.pi/3.0, 20))

ball_state = np.zeros((steps, 4))
ball_state[0, 0] = 5.0
ball_state[0, 1] = 0.0

desired_width = abs(0.0 - ball_state[0,0])
desired_apex = 5.0
desired_theta = np.arctan(4.0 * desired_apex / desired_width)
desired_vlaunch = np.sqrt(2.0*9.81*desired_apex / np.sin(desired_theta)**2.0)
desired_launch_vec = desired_vlaunch * np.array([np.cos(desired_theta), np.sin(desired_theta)])
desired_launch_vec[0] *= np.sign(0.0 - ball_state[0,0])

print "desired_theta, ", desired_theta
print "desired_vlaunch, ", desired_vlaunch
print "desired_launch_vec, ", desired_launch_vec

ball_state[0, 2] = desired_launch_vec[0]
ball_state[0, 3] = desired_launch_vec[1]

# lastErr_x = 0.0
# lastErr_y = 0.0
# lastErr_th = 0.0
# limit = 0.45
collision_thresh = 0.05

# desired_angle = 0.0

# input_u = np.array([2.2, 0.0])

width = 0.4

# Perform an initial trajectory optimization
# quad_goal = np.zeros(6)
# quad_goal[0] = 0.0
# quad_goal[1] = 0.0
# quad_goal[4] = 4.0

# Perform an initial trajectory optimization
quad_traj, input_traj, time_array = plan_trajectory(quad_state[0,:], ball_state[0,:], None)
input_interp = TimeInterpolator(time_array, input_traj, steps, dt)
quad_interp = TimeInterpolator(time_array, quad_traj, steps, dt)

for i in range(0,steps-1):
    # Check for contact
    contact = False
    contact_dir = np.array([-np.sin(quad_state[i,2]), np.cos(quad_state[i,2])])
    pos_diff = ball_state[i,0:2] - quad_state[i,0:2]
    tang = np.array([[0.0, 1.0], [-1.0, 0.0]]).dot(contact_dir)
    tang_comp = tang * pos_diff.dot(tang) if abs(np.linalg.norm(pos_diff)) > 1e-6 else np.zeros(2)
    norm_comp = contact_dir * pos_diff.dot(contact_dir) if abs(np.linalg.norm(pos_diff)) > 1e-6 else np.zeros(2)
    if np.linalg.norm(norm_comp) < collision_thresh and np.linalg.norm(tang_comp) <= width / 2.0:
        contact = True


    contact = sw.leads_to_contact(contact)
    if contact:
        print "CONTACT"

    # PD Control
    # error_x = quad_state[i,0] - (ball_state[i-1,0] if i > 0 else 0.0)                                              
    # dE_x = (error_x - lastErr_x) / dt                                             
    # limit = min(0.45 * abs(error_x)/5.0, 0.25)                                    
    # desired_angle = max(min(10000.0*error_x+ 10000.0*dE_x, limit), -limit)        
    # error_y = 0.0 - quad_state[i,1]                                               
    # error_th = desired_angle - quad_state[i,2]                                    
    # dE_y = (error_y - lastErr_y) / dt                                             
    # dE_th = (error_th - lastErr_th) / dt                                          
    # force_command = 0.1*error_y + 0.3*dE_y + 0.981                                
    # roll_command = 4.0*error_th + 1.5*dE_th                                       
    # input_u = np.array([force_command, roll_command])                             
    # lastErr_y = error_y                                                           
    # lastErr_th = error_th
    # lastErr_x = error_x

    # State update
    if i < quad_interp.get_cutoff_index():
        quad_state[i+1,:] = quad_interp[i] 
    else:
        quad_state[i+1,:] = quad.step_dynamics(dt, quad_state[i,:], input_interp[i])
        
    ball_state[i+1,:] = ball.step_dynamics(dt, ball_state[i,:], quad_state[i,:], contact, contact_dir)

quad_state = np.reshape(quad_state[:,0:3], (steps, 3))

a = Animator(time[::downsample], quad_state[::downsample], ball_state[::downsample])


desired_theta,  1.32581766367
desired_vlaunch,  10.2093706956
desired_launch_vec,  [-2.4761361   9.90454441]
arch_width:  5.0
apex:  5.0
travel_time:  2.01927510938
vland:  [-2.4761361  -9.90454441]
theta:  -1.32581766367
Number of decision vars:  320
Solve:  SolutionResult.kInfeasibleConstraints
CONTACT
[-2.4761361   6.13652341]
tang_comp [ 1.92606398  2.10114698]
norm_comp [-4.40220008  4.03537643]
[ 4.55958717 -4.22615748]
